# Table of contents
1. [Setup](#setup)
    1. [Imports](#imports)
    2. [File constants](#file-constants)
    3. [Wildfire information](#wildfire-info)
    4. [File constants](#file-constants)
2. [Data Collection](#data-collection)
3. [Image Processing](#image-processing)
    1. [Wildifre area](#wildfire-area)
    2. [Morphology](#morphology)
4. [Land Cover Classification](#land-cover-classification)
    1. [Land Cover Datasets](#land-cover-datasets)
    2. [Folium map](#folium-map)

<a name="setup"></a>
## Setup

<a name="imports"></a>
### Imports

In [ ]:
import datetime as dt
import json
import os

import ee
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sentinelsat import SentinelAPI
from skimage.morphology import area_closing

import utils.data_collection as dc
import utils.image_processing as ip
import utils.land_coverage as land_c
import utils.plot_folium as pf

In order to use Google's Earth Engine API, you must create a (free) account at [Earth Engine API](https://developers.google.com/earth-engine) and then authenticate once. Your token will be saved so you will not need to do it again anymore.

In [ ]:
while True:
    try:
        authenticated = input("Authenticate with Earth Engine? (y/n): ")
        if authenticated == "y":
            ee.Authenticate()
            break
        elif authenticated == "n":
            break
        else:
            raise ValueError("Invalid input")
    except ValueError:
        print("Please enter 'y' or 'n'.")
ee.Initialize()

Input the name of the location where the wildfire occurred. Examples are:
* Istres
* Montguers
* etc.

In [ ]:
# Name of the place where the fire is located
try:
    FIRE_NAME = input("Name of the fire: ").lower()
except ValueError:
    print("Please enter a valid name.")

# Folder where the JP2 images will be stored
PATH = f'data/{FIRE_NAME}/'
# Path to the folders where the TIFF, PNG, and GIF files will be stored
OUTPUT_FOLDER = f"output/{FIRE_NAME}/"

<a name="wildfire-info"></a>
### Wildfire information

Some previous information is needed to execute this notebook. This information includes:
* the date of the wildfire in format `YYYY-MM-DD`
* a pair of coordinates _latitude_ and _longitude_ inside the wildfire, such as $44.5, 4.0$
* the true area that burned in hectares. This value is easily retrievable in the news

This information must be saved in a JSON file saved as `info_NAME.json`:

{
    "wildifire_date": "YYYY-MM-DD",
    "longitude": XX.XX,
    "latitude": YY.YY,
    "true_area": ZZ.ZZ
}

where the latitude, longitude, and true area are floats (without the ""). An example is given inside [info_var.json](data/info_var.json).

_Note: the name field is not necessary but is given for clarification._

In [ ]:
with open(f"data/info_{FIRE_NAME}.json") as f:
    fire_info = json.load(f)

# Date of the fire
WILDFIRE_DATE = dt.datetime.strptime(fire_info["wildfire_date"], "%Y-%m-%d")
# Coordinates of the fire
LATITUDE, LONGITUDE = fire_info["latitude"], fire_info["longitude"]
# Actual area in hectares that burned. We retrieved the area on the news
TRUE_AREA = fire_info["true_area"]

The GeoJSON file format is a special type of file storing geospatial data, in a similar format to JSON files. More in-depth information is available at [ArcGIS Online](https://doc.arcgis.com/en/arcgis-online/reference/geojson.htm).

A step-by-step guide to creating a GeoJSON file for an area of interest is given inside the [instructions guide](INSTRUCTIONS.md).

Moreover, you will need an (also free) Copernicus Open Access Hub account [here](https://sentinelsat.readthedocs.io/en/latest/index.html). Then save your credentials in a JSON file inside the _secrets_ folder named `sentinel_api_credentials.json`:

{
    "username": "YOUR_USERNAME",
    "password": "YOUR_PASSWORD"
}.

In [ ]:
# Path to the GeoJSON file
GEOJSON_PATH = f"data/geojson_files/{FIRE_NAME}.geojson"
# Path to the JSON file where the Sentinel API credentials are stored
CREDENTIALS_PATH = "secrets/sentinel_api_credentials.json"

<a name="file-constants"></a>
### File constants

* Observation interval:
<p>The observation interval is somewhat arbitrary, but longer ranges allow us to retrieve more images, since they are taken every 5 days. Furthermore, the burned area is visible for a few weeks, even months, after the fire started.</p>

* Resolution:
<p>As of right now, we only use the <i>NDVI</i> of an image for the processing steps. The necessary bands, <code>B08</code> (near-infrared) and <code>B04</code> (red) are available at all resolutions (10, 20, and 60) but for better results we use 10.
Other indexes, such as <i>SWIR</i>, need resolutions 20 or 60 but are not yet implemented.</p>

* Cloud threshold:
<p>We only retrieve the images below this threshold, because otherwise they are mostly no-data images or yield no valuable results.</p>

* Thresholds:
<p>After calculating the difference in NDVI, we reduce the "noise" of the image by thresholding the values. All values below the threshold are set to 0, so that the burned area stands out.</p>

* Samples:
<p>After retrieving the land cover data, we use sample coordinates to create the pie charts.</p>

In [ ]:
# Number of days both before and after the fire to get images
OBSERVATION_INTERVAL = 15
# Resolution of the images (10m, 20m, or 60m)
RESOLUTION = 10
# Threshold for the cloud cover (between 0 and 100)
CLOUD_THRESHOLD = 40
# Seed for random number generator (for reproductibility)
SEED = 42
# Threshold values for calculating the area that burned
THRESHOLDS = np.arange(0.1, 0.41, 0.02)
# Number of coordinates to use for the pie charts
SAMPLES = np.arange(50, 251, 50)

<a name="data-collection"></a>
## Data Collection

In [ ]:
with open(CREDENTIALS_PATH, 'r') as infile:
    credentials = json.load(infile)

api = SentinelAPI(
    credentials["username"],
    credentials["password"]
)

In [ ]:
exists = os.path.exists(PATH)
if not exists:
    os.makedirs(PATH)

This is the main function that retrieves and downloads the relevant images. You may get the error __Product ... is not online. Triggering retrieval from long term archive__, specially with older images. Unfortunately, the only solution we have found is to wait for 15-30 minutes (maybe more) and then try again.

Another error, __NullPointerException__ may occur, but the solution is the same: try again after a few minutes.

In [ ]:
if not dc.check_downloaded_data(PATH, OUTPUT_FOLDER, FIRE_NAME):
    try:
        dc.get_before_after_images(
            api=api,
            wildfire_date=WILDFIRE_DATE,
            geojson_path=GEOJSON_PATH,
            observation_interval=OBSERVATION_INTERVAL,
            path=PATH,
            fire_name=FIRE_NAME,
            output_folder=OUTPUT_FOLDER,
            resolution=RESOLUTION,
            cloud_threshold=CLOUD_THRESHOLD
        )
    except Exception as e:
        print(e)
        exit()

In [ ]:
ip.plot_downloaded_images(FIRE_NAME, OUTPUT_FOLDER)

<a name="image-processing"></a>
## Image Processing

The first step here is to retrieve the location of the fire inside the produced NDVI image. Instead of doing this manually, we can transform the coordinates from the JSON info file into pixel values. Then, we plot the pixels inside the image.

In [ ]:
img_folder = PATH + os.listdir(PATH)[1] + '/'
pixel_column, pixel_row = ip.get_fire_pixels(
    img_folder, LATITUDE, LONGITUDE
)

In [ ]:
print(f'The fire is located at pixels ({pixel_column}, {pixel_row})\n')

In [ ]:
diff = dc.plot_ndvi_difference(
    OUTPUT_FOLDER, FIRE_NAME, figsize=(10, 10)
)
plt.plot(pixel_column, pixel_row, 'ro',
         markersize=3, label='Fire Location')
plt.legend()
plt.show()

Next, we ask the user to give pixel values to add lines to the image to zoom-in on the area of interest. We have explored methods to do this manually but we have not been successful for now.

In [ ]:
fire, hline_1, vline_1 = ip.retrieve_fire_area(
    diff, pixel_column, pixel_row, 'Fire Area'
)

<a name="wildifre-area"></a>
### Wildfire area

Here we calculate the area that burned thanks to the differnce in NDVI and we plot the results, along with the true value. This helps us to validate our functions.

In [ ]:
areas = []
for thr in THRESHOLDS:
    tmp = ip.threshold_filter(fire, thr)
    area = round(ip.calculate_area(tmp, diff) * 100, 4)
    areas.append(area)

In [ ]:
print(f'The true area that burned is {TRUE_AREA} hectares.\n')

In [ ]:
plt.figure(figsize=(8, 6))
with sns.axes_style('darkgrid'):
    plt.plot(THRESHOLDS, areas)
    plt.hlines(TRUE_AREA, THRESHOLDS[0], THRESHOLDS[-1],
               colors='r', linestyles='dashed')
    plt.xlabel('Threshold')
    plt.ylabel('Burnt Area (ha)')
    plt.title('Fire Area')
    plt.legend(['Calculated Area', 'True Value'])
    plt.show()

In [ ]:
while True:
    threshold = float(input('Enter a threshold value between -1 and 1: '))
    try:
        if -1 <= threshold <= 1:
            break
        else:
            print('Please enter a value between -1 and 1.')
    except ValueError:
        print('Please enter a valid number.')

fire = ip.threshold_filter(fire, threshold)
plt.figure(figsize=(8, 6))
ip.imshow(tmp, 'Thresholded Fire')
plt.show()

In [ ]:
print('Area:', round(ip.calculate_area(fire, diff) * 100, 4), 'ha.')

<a name="morphology"></a>
### Morphology

Finally, for the image processing part, we use mathematical morphology to slighlty improve the quality of the image. More details are available [here](https://scikit-image.org/docs/stable/api/skimage.morphology.html).

In [ ]:
closed = area_closing(fire)
ip.plot_comparison(fire, closed, 'Area Closing')

In [ ]:
print('Area after morphology:', round(ip.calculate_area(closed, diff) * 100, 4), 'ha.')

<a name="land-cover-classification"></a>
## Land Cover Classification

<a name="land-cover-datasets"></a>
### Land cover datasets

The next step consists in retrieving information on the type of land that was affected by the fire, such as crops or forests. Multiple datasets are available from Earth Engine's catalog. They use different resolutions and time ranges. The ones we use are:
* [MODIS Land Cover](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1?hl=en) _(2018, 500m)_
* [ESA World Cover](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100) _(2020, 10m)_
* [Copernicus Global Land Service](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global) _(2019, 100m)_
* [Copernicus CORINE Land Cover](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_CORINE_V20_100m) _(2018, 100m)_

In [ ]:
while True:
    try:
        prob = input(
            r'Enter sample percentage to use for land cover classification: '
        )
        p = float(p) / 100
        if 0 < p <= 1:
            break
        else:
            print('Please enter a value between 0 and 100.')
    except ValueError:
        print('Please enter a valid number.')

rand_image = land_c.create_sample_coordinates(fire, SEED, prob)
ip.imshow(rand_image, 'Sampled Coordinates')
plt.show()

Conversely, we can transform pixel values from the image into coordinates.

In [ ]:
coordinates = land_c.get_coordinates_from_pixels(
    rand_image, hline_1, vline_1, img_folder, FIRE_NAME
)

In [ ]:
output_folder = f"output/pie_chart_{FIRE_NAME}/"
exists = os.path.exists(output_folder)
if not exists:
    os.makedirs(output_folder)
is_empty = not any(os.scandir(output_folder))

After asking the user for a land cover dataset, we create a GIF file from the pie charts, to see the evolution of land cover information we obtain.

In [ ]:
choice = land_c.get_choice()

In [ ]:
if exists and is_empty or not exists:
    land_c.create_plots(
        samples=SAMPLES,
        coordinates=coordinates,
        choice=choice,
        seed=SEED,
        fire_name=FIRE_NAME,
        out_folder=output_folder,
        save_fig=True
    )

In [ ]:
land_c.make_pie_chart_gif(
    fire_name=FIRE_NAME,
    file_path=output_folder,
    save_all=True,
    duration=500,
    loop=0
)

<a name="folium-map"></a>
### Create folium map

Finally, we create an interactive map using `folium` to visualize the coordinates of the fire in a Google Satellite map, also adding the land cover layer. Then we open the produced GIF.

In [ ]:
while True:
    try:
        prob = float(input('Value between 0 and 100: ')) / 100
        if 0 < p <= 1:
            break
        else:
            raise ValueError
    except ValueError:
        print('Please enter a valid number.')

fire_map = pf.create_map(FIRE_NAME, prob, SEED, choice)

In [ ]:
output_maps = "output/maps/"
if not os.path.exists(output_maps):
    os.makedirs(output_maps)

pf.save_map(fire_map, FIRE_NAME, output_maps)
pf.open_map(FIRE_NAME, output_maps)

In [ ]:
land_c.open_gif(FIRE_NAME, output_folder)

<a name="wind-data"></a>
## Wind Data